In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
import pathlib
import pickle
from pathlib import Path
from skimage.io import imread
import seaborn as sns
sns.set_style('white')
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.utils.plot_funcs import plot_od, plot_train_record

In [4]:
model_path = Path('/mnt/sda1/SMLAT/training_runs/model_rotated_45_venus_beads.pkl')

In [5]:
with open(model_path, 'rb') as f:
    chromo_model = pickle.load(f)

In [6]:
chromo_model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [7]:
plot_train_record(chromo_model)

#### Photon count range, get background average from simulation using model paramteres

In [ ]:
mol_photons = (chromo_model.data_generator.simulation_params['min_photon'] + 1) /2 * chromo_model.data_generator.psf_params['photon_scale']

In [ ]:
mol_photons

#### Plot network outputs for the eval images after tiling and re-tiling of one eval image

In [8]:
from cellbgnet.analyze_eval import recognition, plot_full_img_predictions, assemble_full_img_predictions

In [9]:
eval_img = chromo_model.evaluation_params['eval_imgs']

In [10]:
eval_img.shape

(30, 1041, 1302)

In [11]:
plt.figure()
plt.imshow(eval_img[0], cmap='gray')
plt.colorbar()
plt.show()

In [12]:
fov_size = [eval_img.shape[2] * 65, eval_img.shape[1] * 65]

In [13]:
fov_size

[84630, 67665]

In [14]:
eval_img.mean(0).max()

129.0405049641927

In [15]:
eval_img.mean()

113.72014866131897

In [ ]:
preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=eval_img,
                                             batch_size=16, use_tqdm=False,
                                             nms=True, candidate_threshold=0.05,
                                             nms_threshold=0.05, 
                                             pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                             plot_num=1,
                                             win_size=128,
                                             padding=True,
                                             start_field_pos=[0, 0],
                                             padded_background=chromo_model.evaluation_params['padded_background'])

In [ ]:
plot_full_img_predictions(chromo_model, plot_infs=plot_data, eval_csv=None, plot_num=1, fov_size=fov_size, pixel_size=[65, 65])

In [ ]:
img_infs = assemble_full_img_predictions(chromo_model, plot_data)

In [ ]:
img_infs.keys()

In [16]:
from scipy.spatial.distance import cdist
from cellbgnet.analyze_eval import limited_matching

In [ ]:
ground_truth = chromo_model.evaluation_params['ground_truth']

In [ ]:
perf_dict, matches = limited_matching(ground_truth, preds_tmp, min_int=0, limited_x=[0, fov_size[0]],
                                     limited_y=[0, fov_size[1]], border=450, print_res=True, tolerance=250,
                                      tolerance_ax=np.inf)

#### Testing real image

In [17]:
import skimage.io as sio

In [18]:
images_dir = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4713/fluorChannelRegistration3D/zStack_locError_101/Pos100/')
img_filenames = sorted(list(images_dir.glob('*.tif')))

In [19]:
eval_imgs_all = []
for filename in img_filenames:
    eval_imgs_all.append(sio.imread(filename))

In [20]:
eval_imgs_all = np.stack(eval_imgs_all)

In [21]:
eval_imgs_all = eval_imgs_all[:, 1041:, :]

In [22]:
eval_imgs_all.shape

(50, 1041, 1302)

In [23]:
fov_size = [eval_imgs_all.shape[2] * 65, eval_imgs_all.shape[1] * 65]

In [24]:

preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=eval_imgs_all,
                                             batch_size=16, use_tqdm=False,
                                             nms=True, candidate_threshold=0.4,
                                             nms_threshold=0.5, 
                                             pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                             plot_num=1,
                                             win_size=128,
                                             padding=True,
                                             start_field_pos=[0, 0],
                                             padded_background=chromo_model.evaluation_params['padded_background'])


processing area:1/99, input field_xy:[ -2 145  -3 144], use_coordconv:True, retain locs in area:[-2, 125, -3, 124]

/home/pk/Documents/cellbgnet/cellbgnet/utils/hardware.py:9: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  return torch.cuda.FloatTensor(x.astype('float32'))


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]


In [25]:
img_infs = assemble_full_img_predictions(chromo_model, plot_data)
img_infs.keys()

dict_keys(['Probs', 'XO', 'YO', 'ZO', 'Int', 'BG', 'XO_sig', 'YO_sig', 'ZO_sig', 'Int_sig', 'Probs_ps', 'XO_ps', 'YO_ps', 'ZO_ps', 'Samples_ps', 'raw_img', 'only_bg'])

### Matching emitters based on dist

In [26]:

preds_tmp_np = np.array(preds_tmp)

In [27]:
frame_wise_xyz = [preds_tmp_np[np.where(preds_tmp_np[:, 1] == i)].tolist() for i in range(1, 1 + eval_imgs_all.shape[0])]

In [28]:
len(frame_wise_xyz)

50

### For emitter in first frame, find the closest emitter in all the other frames < 250 nm distance from it

In [29]:
from scipy.spatial.distance import cdist
from cellbgnet.analyze_eval import limited_matching

In [30]:
first_frame_localizations = frame_wise_xyz[0]

In [31]:
first_frame_localizations

[[1.0,
  1.0,
  46017.7890625,
  22184.791015625,
  520.5870361328125,
  1428.6990966796875,
  0.8036450147628784,
  13.731034278869629,
  13.318632125854492,
  27.192838668823242,
  116.68636322021484,
  -0.0340123288333416,
  0.30447155237197876],
 [1.0,
  1.0,
  72675.421875,
  27667.8125,
  400.9862365722656,
  2017.6279296875,
  0.9052897691726685,
  11.581977844238281,
  10.729774475097656,
  41.721927642822266,
  130.83038330078125,
  0.0833640769124031,
  -0.34133994579315186],
 [1.0,
  1.0,
  76574.953125,
  50609.03125,
  383.5773620605469,
  1236.3638916015625,
  0.8059327602386475,
  15.630094528198242,
  14.882637023925781,
  57.81389617919922,
  111.07596588134766,
  0.07619024068117142,
  -0.3995388150215149]]

In [32]:
frame_wise_xyz[1]

[[2.0,
  2.0,
  46005.71875,
  22167.822265625,
  467.903076171875,
  2000.3468017578125,
  0.934842050075531,
  11.831757545471191,
  11.39588737487793,
  41.918087005615234,
  148.72128295898438,
  -0.21971970796585083,
  0.043434370309114456],
 [2.0,
  2.0,
  72660.390625,
  27650.208984375,
  563.8120727539062,
  1263.6446533203125,
  0.8924051523208618,
  13.646117210388184,
  13.189980506896973,
  18.85953140258789,
  106.85687255859375,
  -0.14780175685882568,
  0.3878340721130371],
 [2.0,
  2.0,
  76572.2734375,
  50569.12109375,
  440.40240478515625,
  1410.6142578125,
  0.7850717306137085,
  14.049369812011719,
  13.41433334350586,
  45.783084869384766,
  116.94346618652344,
  0.03494017943739891,
  -0.013513751327991486]]

In [33]:
perf_dict, matches = match_two_frames(frame_wise_xyz[0], frame_wise_xyz[1], min_int=0, limited_x=[0, fov_size[0]],
                             limited_y=[0, fov_size[1]], border=450, print_res=True, tolerance=450,
                             tolerance_ax=np.inf)

NameError: name 'match_two_frames' is not defined

In [34]:
import copy

In [35]:

def match_two_frames(truth_origin, pred_list_origin, min_int, limited_x=[0, 204800], limited_y=[0, 204800],
                     border=450, print_res=True, tolerance=250, tolerance_ax=np.inf):
    #print('{}{}{}{}'.format('FOV: x=', limited_x, ' y=', limited_y))

    matches = []

    truth = copy.deepcopy(truth_origin)
    pred_list = copy.deepcopy(pred_list_origin)

    truth_array = np.array(truth)
    pred_array = np.array(pred_list)
    print(f"Pred_array shape: {pred_array.shape}")

    if len(pred_list) == 0:
        perf_dict = {'recall': np.nan, 'precision': np.nan, 'jaccard': np.nan, 'f_score': np.nan, 'rmse_lat': np.nan,
                     'rmse_ax': np.nan,
                     'rmse_x': np.nan, 'rmse_y': np.nan, 'jor': np.nan, 'eff_lat': np.nan, 'eff_ax': np.nan,
                     'eff_3d': np.nan}
        print('after FOV segmentation, pred_list is empty!')
        return perf_dict, matches
    print('{}{}{}{}{}'.format('after FOV and border segmentation,', 'truth: ', len(truth), ' ,preds: ', len(pred_list)))
    # delete molecules of ground truth/estimation in the margin area
    

    # filter prediction and gt according to limited_x;y
    t_inds = np.where(
        (truth_array[:, 2] < limited_x[0]) | (truth_array[:, 2] > limited_x[1]) |
        (truth_array[:, 3] < limited_y[0]) | (truth_array[:, 3] > limited_y[1]))
    p_inds = np.where(
        (pred_array[:, 2] < limited_x[0]) | (pred_array[:, 2] > limited_x[1]) |
        (pred_array[:, 3] < limited_y[0]) | (pred_array[:, 3] > limited_y[1]))
    for t in reversed(t_inds[0]):
        del (truth[t])
    for p in reversed(p_inds[0]):
        del (pred_list[p])

    if border:
        test_arr = np.array(truth)
        pred_arr = np.array(pred_list)

        t_inds = np.where(
            (test_arr[:, 2] < limited_x[0] + border) | (test_arr[:, 2] > (limited_x[1] - border)) |
            (test_arr[:, 3] < limited_y[0] + border) | (test_arr[:, 3] > (limited_y[1] - border)))
        p_inds = np.where(
            (pred_arr[:, 2] < limited_x[0] + border) | (pred_arr[:, 2] > (limited_x[1] - border)) |
            (pred_arr[:, 3] < limited_y[0] + border) | (pred_arr[:, 3] > (limited_y[1] - border)))
        for t in reversed(t_inds[0]):
            del (truth[t])
        for p in reversed(p_inds[0]):
            del (pred_list[p])

    if len(pred_list) == 0:
        perf_dict = {'recall': np.nan, 'precision': np.nan, 'jaccard': np.nan, 'f_score': np.nan, 'rmse_lat': np.nan,
                     'rmse_ax': np.nan,
                     'rmse_x': np.nan, 'rmse_y': np.nan, 'jor': np.nan, 'eff_lat': np.nan, 'eff_ax': np.nan,
                     'eff_3d': np.nan}
        print('after border, pred_list is empty!')
        return perf_dict, matches

    print('{}{}{}{}{}'.format('after FOV and border segmentation,', 'truth: ', len(truth), ' ,preds: ', len(pred_list)))

    TP = 0
    FP = 0.0001
    FN = 0.0001
    MSE_lat = 0
    MSE_ax = 0
    MSE_vol = 0

    if len(pred_list):
        print(f"Length of preds_list: {len(pred_list)}")
        tests = copy.deepcopy(truth)  # gt in each frame
        preds = copy.deepcopy(pred_list)  # prediction in each frame

        #if len(truth) > 0:  # after border filtering and area segmentation, truth could be empty
        #    while truth[0][1] == i:
        #        tests.append(truth.pop(0))  # put all gt in the tests
        #        if len(truth) < 1:
        #            break
        #if len(pred_list) > 0:
        #    while pred_list[0][1] == i:
        #        preds.append(pred_list.pop(0))  # put all predictions in the preds
        #        if len(pred_list) < 1:
        #            break
        #print(len(tests), len(preds))
        # if preds is empty, it means no detection on the frame, all tests are FN
        if len(preds) == 0:
            FN += len(tests)
            # no need to calculate metric
        # if the gt of this frame is empty, all preds on this frame are FP
        if len(tests) == 0:
            FP += len(preds)
            # no need to calculate metric
        # calculate the Euclidean distance between all gt and preds, get a matrix [number of gt, number of preds]
        dist_arr = cdist(np.array(tests)[:, 2:4], np.array(preds)[:, 2:4])
        ax_arr = cdist(np.array(tests)[:, 4:5], np.array(preds)[:, 4:5])
        tot_arr = np.sqrt(dist_arr ** 2 + ax_arr ** 2)
        if tolerance_ax == np.inf:
            tot_arr = dist_arr

        match_tests = copy.deepcopy(tests)
        match_preds = copy.deepcopy(preds)
        #print(dist_arr)
        if dist_arr.size > 0:
            while dist_arr.min() < tolerance:
                r, c = np.where(tot_arr == tot_arr.min())  # select the positions pair with shortest distance
                r = r[0]
                c = c[0]
                if ax_arr[r, c] < tolerance_ax and dist_arr[r, c] < tolerance:  # compare the distance and tolerance
                    if match_tests[r][5] > min_int:  # photons should be larger than min_int

                        MSE_lat += dist_arr[r, c] ** 2
                        MSE_ax += ax_arr[r, c] ** 2
                        MSE_vol += dist_arr[r, c] ** 2 + ax_arr[r, c] ** 2
                        TP += 1
                        matches.append([match_tests[r][2], match_tests[r][3], match_tests[r][4], match_tests[r][5],
                                        match_preds[c][2], match_preds[c][3], match_preds[c][4], match_preds[c][5],
                                        match_preds[c][7], match_preds[c][8], match_preds[c][9],
                                        match_preds[c][10]])

                    dist_arr[r, :] = np.inf
                    dist_arr[:, c] = np.inf
                    tot_arr[r, :] = np.inf
                    tot_arr[:, c] = np.inf

                    tests[r][5] = -100  # photon cannot be negative, work as a flag
                    preds.pop()

                dist_arr[r, c] = np.inf
                tot_arr[r, c] = np.inf
                #print("matched one")

        for i in reversed(range(len(tests))):
            if tests[i][5] < min_int:  # delete matched gt
                del (tests[i])

        FP += len(preds)  # all remaining preds are FP
        FN += len(tests)  # all remaining gt are FN
    else:
        print('after border and FOV segmentation, pred list is empty!')

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    jaccard = TP / (TP + FP + FN)
    rmse_lat = np.sqrt(MSE_lat / (TP + 0.00001))
    rmse_ax = np.sqrt(MSE_ax / (TP + 0.00001))
    rmse_vol = np.sqrt(MSE_vol / (TP + 0.00001))
    jor = 100 * jaccard / rmse_lat

    eff_lat = 100 - np.sqrt((100 - 100 * jaccard) ** 2 + 1 ** 2 * rmse_lat ** 2)
    eff_ax = 100 - np.sqrt((100 - 100 * jaccard) ** 2 + 0.5 ** 2 * rmse_ax ** 2)
    eff_3d = (eff_lat + eff_ax) / 2

    matches = np.array(matches)
    #print("Number of matches: ", matches.shape)
    rmse_x = np.nan
    rmse_y = np.nan
    rmse_z = np.nan
    rmse_i = np.nan
    if len(matches):
        rmse_x = np.sqrt(((matches[:, 0] - matches[:, 4]) ** 2).mean())
        rmse_y = np.sqrt(((matches[:, 1] - matches[:, 5]) ** 2).mean())
        rmse_z = np.sqrt(((matches[:, 2] - matches[:, 6]) ** 2).mean())
        rmse_i = np.sqrt(((matches[:, 3] - matches[:, 7]) ** 2).mean())
    else:
        print('matches is empty!')

    if print_res:
        print('{}{:0.3f}'.format('Recall: ', recall))
        print('{}{:0.3f}'.format('Precision: ', precision))
        print('{}{:0.3f}'.format('Jaccard: ', 100 * jaccard))
        print('{}{:0.3f}'.format('RMSE_lat: ', rmse_lat))
        print('{}{:0.3f}'.format('RMSE_ax: ', rmse_ax))
        print('{}{:0.3f}'.format('RMSE_vol: ', rmse_vol))
        print('{}{:0.3f}'.format('Jaccard/RMSE: ', jor))
        print('{}{:0.3f}'.format('Eff_lat: ', eff_lat))
        print('{}{:0.3f}'.format('Eff_ax: ', eff_ax))
        print('{}{:0.3f}'.format('Eff_3d: ', eff_3d))
        print('FN: ' + str(np.round(FN)) + ' FP: ' + str(np.round(FP)))

    perf_dict = {'recall': recall, 'precision': precision, 'jaccard': jaccard, 'rmse_lat': rmse_lat,
                 'rmse_ax': rmse_ax, 'rmse_vol': rmse_vol, 'rmse_x': rmse_x, 'rmse_y': rmse_y,
                 'rmse_z': rmse_z, 'rmse_i': rmse_i, 'jor': jor, 'eff_lat': eff_lat, 'eff_ax': eff_ax,
                 'eff_3d': eff_3d}

    return perf_dict, matches



### Match first bead

In [36]:
bead_data = []

In [37]:
for beadno in range(len(frame_wise_xyz[0])):
    single_bead_data = {}
    single_bead_data['bead_no'] = beadno
    single_bead_data['x'] = [frame_wise_xyz[0][beadno][2]]
    single_bead_data['y'] = [frame_wise_xyz[0][beadno][3]]
    single_bead_data['z'] = [frame_wise_xyz[0][beadno][4]]
    single_bead_data['ph'] = [frame_wise_xyz[0][beadno][5]]
    single_bead_data['x_sigma'] = [frame_wise_xyz[0][beadno][7]]
    single_bead_data['y_sigma'] = [frame_wise_xyz[0][beadno][8]]
    single_bead_data['z_sigma'] = [frame_wise_xyz[0][beadno][9]]
    single_bead_data['ph_sigma'] = [frame_wise_xyz[0][beadno][10]]
    for i in range(1, eval_imgs_all.shape[0]):
        perf_dict, matches = match_two_frames([frame_wise_xyz[0][beadno]], frame_wise_xyz[i], min_int=0, limited_x=[0, fov_size[0]],
                                     limited_y=[0, fov_size[1]], border=450, print_res=False, tolerance=450,
                                     tolerance_ax=np.inf)
        if len(matches) == 1:
            single_bead_data['x'].append(matches[0, 4])
            single_bead_data['y'].append(matches[0, 5])
            single_bead_data['z'].append(matches[0, 6])
            single_bead_data['ph'].append(matches[0, 7])
            single_bead_data['x_sigma'].append(matches[0, 8])
            single_bead_data['y_sigma'].append(matches[0, 9])
            single_bead_data['z_sigma'].append(matches[0, 10])
            single_bead_data['ph_sigma'].append(matches[0, 11])
            #print(f"First bead x: {matches[:, 0]} -- matched: {matches[:, 4]}")
            #print(f"First bead y: {matches[:, 1]} -- matched: {matches[:, 5]}")
            #print(f"First bead z: {matches[:, 2]} -- matched: {matches[:, 6]}")
            #print(f"First bead ph: {matches[:, 3]} -- matched: {matches[:, 7]}")
            #print("---------")
        else:
            print(f"For beadno: {beadno} --- Skipping frame {i} --- :(")
    bead_data.append(single_bead_data)

Pred_array shape: (3, 13)
after FOV and border segmentation,truth: 1 ,preds: 3
after FOV and border segmentation,truth: 1 ,preds: 3
Length of preds_list: 3
Pred_array shape: (2, 13)
after FOV and border segmentation,truth: 1 ,preds: 2
after FOV and border segmentation,truth: 1 ,preds: 2
Length of preds_list: 2
Pred_array shape: (3, 13)
after FOV and border segmentation,truth: 1 ,preds: 3
after FOV and border segmentation,truth: 1 ,preds: 3
Length of preds_list: 3
Pred_array shape: (1, 13)
after FOV and border segmentation,truth: 1 ,preds: 1
after FOV and border segmentation,truth: 1 ,preds: 1
Length of preds_list: 1
matches is empty!
For beadno: 0 --- Skipping frame 4 --- :(
Pred_array shape: (3, 13)
after FOV and border segmentation,truth: 1 ,preds: 3
after FOV and border segmentation,truth: 1 ,preds: 3
Length of preds_list: 3
matches is empty!
For beadno: 0 --- Skipping frame 5 --- :(
Pred_array shape: (4, 13)
after FOV and border segmentation,truth: 1 ,preds: 4
after FOV and border 

/tmp/ipykernel_130468/2745406847.py:147: RuntimeWarning: invalid value encountered in scalar divide
  jor = 100 * jaccard / rmse_lat


In [38]:
bead_data[2]['z']

[383.5773620605469,
 440.40240478515625,
 398.3832092285156,
 485.3009033203125,
 404.6663513183594,
 411.6457824707031,
 354.3451843261719,
 412.8780212402344,
 396.7459716796875,
 431.0600891113281,
 444.0550537109375,
 472.4336242675781,
 325.8613586425781]

In [39]:
len(bead_data[0]['x'])

38

In [40]:
bead_data[0].keys()

dict_keys(['bead_no', 'x', 'y', 'z', 'ph', 'x_sigma', 'y_sigma', 'z_sigma', 'ph_sigma'])

In [41]:
def plot_bead_data(single_bead_data):
    x_values = np.array(single_bead_data['x']) - np.mean(single_bead_data['x'])
    y_values = np.array(single_bead_data['y']) - np.mean(single_bead_data['y'])
    z_values = np.array(single_bead_data['z']) - np.mean(single_bead_data['z'])
    ph_values = np.array(single_bead_data['ph'])
    fig, ax = plt.subplots(nrows=2, ncols=4)
    ax[0, 0].hist(single_bead_data['x'] - np.mean(single_bead_data['x']), bins=40, density=True)
    ax[0, 0].axvspan(np.min(x_values), np.max(x_values), color='green', alpha=0.1)
    ax[0, 0].set_title(f"x localizations n = {len(x_values)}")
    ax[0, 0].set_xlabel('nm')
    
    ax[0, 1].hist(single_bead_data['y'] - np.mean(single_bead_data['y']), bins=40, density=True)
    ax[0, 1].axvspan(np.min(y_values), np.max(y_values), color='green', alpha=0.1)
    ax[0, 1].set_title(f"y localizations n = {len(y_values)}")
    ax[0, 1].set_xlabel('nm')

    ax[0, 2].hist(single_bead_data['z'] - np.mean(single_bead_data['z']), bins=40, density=True)
    ax[0, 2].axvspan(np.min(z_values), np.max(z_values), color='green', alpha=0.1)
    ax[0, 2].set_title(f"z localizations n = {len(z_values)}")
    ax[0, 2].set_xlabel('nm')

    ax[0, 3].hist(single_bead_data['ph'], bins=40, density=True)
    ax[0, 3].axvspan(np.min(ph_values), np.max(ph_values), color='green', alpha=0.1)
    ax[0, 3].set_title(f"ph n = {len(ph_values)}")
    ax[0, 3].set_xlabel('counts')
    
    
    ax[1, 0].hist(np.array(single_bead_data['x_sigma']), bins=40, density=True)
    ax[1, 0].axvspan(min(single_bead_data['x_sigma']), max(single_bead_data['x_sigma']), color='green', alpha=0.1)
    ax[1, 0].set_title(f"x sigma net predicted n = {len(x_values)}")
    ax[1, 0].set_xlabel('nm')

    ax[1, 1].hist(single_bead_data['y_sigma'], bins=40, density=True)
    ax[1, 1].axvspan(min(single_bead_data['y_sigma']), max(single_bead_data['y_sigma']), color='green', alpha=0.1)
    ax[1, 1].set_title(f"y sigma net predicted n = {len(y_values)}")
    ax[1, 1].set_xlabel('nm')

    ax[1, 2].hist(single_bead_data['z_sigma'], bins=40, density=True)
    ax[1, 2].axvspan(min(single_bead_data['z_sigma']), max(single_bead_data['z_sigma']), color='green', alpha=0.1)
    ax[1, 2].set_title(f"z sigma net predicted n = {len(z_values)}")
    ax[1, 2].set_xlabel('nm')

    ax[1, 3].hist(single_bead_data['ph_sigma'], bins=40, density=True)
    ax[1, 3].axvspan(np.min(single_bead_data['ph_sigma']), np.max(single_bead_data['ph_sigma']), color='green', alpha=0.1)
    ax[1, 3].set_title(f"ph sigma n = {len(ph_values)}")
    ax[1, 3].set_xlabel('nm')

    
    plt.tight_layout()
    plt.suptitle("One bead")
    plt.show()

In [45]:
len(bead_data)

3

In [46]:
plot_bead_data(bead_data[2])

In [49]:
for i in range(len(bead_data)):
    plot_bead_data(bead_data[i])

#### Plot data from all beads in one plot

#### Predictions are a list of numbers where each element corresponds to a localizaiton in the following order


    1. counter of the molecule per tile.. not a global counter on the frame, need to fix this
    2. image number used to index into the number of the image in the prediction arrays.
    Eval image is of shape [1, 1041, 1302], so, the image number will always be 1, if you give one image at a time
    3. x position in nm where 0 is top left corner
    4. y position in nm where 0 is top left corner
    5. z position in nm where 0 is from the reference 0 nm in height
    6. photon counts 
    7. probability afer nms
    8. x_sigma in nm 
    9. y_sigma in nm
    10. z_sigma in nm
    11. photon_counts_sigma
    12. x offset
    13. y offset
